<a href="https://colab.research.google.com/github/Armstrong66/Armstrong66/blob/main/IQMPulseViewer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Compile quantum circuits into pulses
In this first exercise, you will get an idea of unitary gate implementation through direct visualization of pulses.

In this notebook, you will ...
* Prepare a GHZ state and compile the corresponding quantum circuit into a *playlist* of microwave pulses.

By the end of this notebook, you will understand the correspondence between unitary gates and their physical implementation for superconducting qubits.

In order to get started, make sure you have the appropriate packages installed:

In [ ]:
!pip install pip==23.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 27.6 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


In [ ]:
# %%capture
!pip install iqm-client==32.1.1
!pip install iqm-station-control-client==11.3.1
!pip install iqm-pulla==11.16.2
!pip install iqm-pulse==12.6.1

In [ ]:
!pip install qrisp[iqm]

  Using cached iqm_client-32.1.1-py3-none-any.whl.metadata (17 kB)
  Using cached iqm_station_control_client-11.3.1-py3-none-any.whl.metadata (14 kB)
  Using cached packaging-24.1-py3-none-any.whl.metadata (3.2 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 6.4 MB/s eta 0:00:00
  Using cached opentelemetry_exporter_otlp-1.39.0-py3-none-any.whl.metadata (2.4 kB)
  Using cached types_tqdm-4.67.0.20250809-py3-none-any.whl.metadata (1.7 kB)
  Using cached opentelemetry_exporter_otlp_proto_grpc-1.39.0-py3-none-any.whl.metadata (2.5 kB)
  Using cached opentelemetry_exporter_otlp_proto_http-1.39.0-py3-none-any.whl.metadata (2.4 kB)
  Using cached opentelemetry_api-1.39.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached opentelemetry_exporter_otlp_proto_common-1.39.0-py3-none-any.whl.metadata (1.8 kB)
  Using cached opentelemetry_proto-1.39.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached opentelemetry_sdk-1.39.0-py3-none-any.whl.metadata (1.5 kB)
INFO: pip is looking at mu

## 1. Preparation

### 1.1 Connecting to the QPU station control
In the following, we will use the PulLa (Pulse Level Access) package. You can find the documentation [here](https://docs.meetiqm.com/iqm-pulla/).

As a first step, we need to create a **PulLa object**. In general, this is a **compiler**, in a particular state, linked to a particular quantum computer. It contains calibration data and the set of available operations.

Make sure you have the correct url and token below.

In [ ]:
from iqm.pulla.pulla import Pulla

api_token = input()
p = Pulla("https://cocos.resonance.meetiqm.com/garnet", get_token_callback=lambda: api_token)

compiler = p.get_standard_compiler()

ModuleNotFoundError: No module named 'iqm.pulla'

## 2. GHZ state

You have learnt in the past labs and exercises what a GHZ state is and how to prepare one on a quantum computer. Here, we are not interested in its entanglement properties, but we will use it as a starting point to look into physical pulses. Let's construct a GHZ for 3 qubits:

In [ ]:
from qrisp import QuantumCircuit

qc_0 = #TODO

print(qc_0)

Before compiling the circuit, we need to transpile it to take into account the native gates and connectivity of IQM quantum computers:

In [ ]:
from iqm.qiskit_iqm import IQMProvider

provider = IQMProvider('https://cocos.resonance.meetiqm.com/garnet', token=api_token)
backend = provider.get_backend()

transpiled_qc = #TODO
print(transpiled_qc)

To use the PulLA visualization tool, we need to convert the qrisp circuit into a qiskit one. We use the converter as below:

In [ ]:
qiskit_circuit = transpiled_qc.to_qiskit()

## 3. Compilation

We can now extract the list of *instructions* from the circuit defined above and compile it to obtain a *playlist* of pulses!

In [ ]:
from iqm.qiskit_iqm.qiskit_to_iqm import serialize_instructions
from iqm.iqm_client import Circuit

iqm_instructions = serialize_instructions(qiskit_circuit, {i : "QB" + str(i+1) for i in range(qiskit_circuit.num_qubits)})
iqm_circuit = Circuit(name = "my_circuit", instructions = iqm_instructions)

playlist, context = #TODO

Finally, we are ready to visualize the pulses that prepare a GHZ state of 3 qubits. Let's explore the details:

In [ ]:
from iqm.pulse.playlist.visualisation.base import inspect_playlist
from IPython.core.display import HTML

HTML(inspect_playlist(playlist))